# Tuning and Optimizing Neural Networks - Lab

## Introduction

Now that you've practiced regularization, initialization, and optimization techniques, its time to synthesize these concepts into a cohesive modeling pipeline.  

With this pipeline, you will not only fit an initial model but also attempt to improve it. Your final model selection will pertain to the test metrics across these models. This will more naturally simulate a problem you might be faced with in practice, and the various modeling decisions you are apt to encounter along the way.  

Recall that our end objective is to achieve a balance between overfitting and underfitting. You've seen the bias variance trade-off, and the role of regularization in order to reduce overfitting on training data and improving generalization to new cases. Common frameworks for such a procedure include train/validate/test methodology when data is plentiful, and K-folds cross-validation for smaller, more limited datasets. In this lab, you'll perform the latter, as the dataset in question is fairly limited. 

## Objectives

You will be able to:

* Apply normalization as a preprocessing technique 
* Implement a K-folds cross validation modeling pipeline for deep learning models 
* Apply regularization techniques to improve your model's performance 

## Load the data

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from keras import models
from keras import layers
from keras import regularizers
from keras.wrappers.scikit_learn import KerasRegressor

Using TensorFlow backend.


In [2]:
pd.set_option('max_columns', None)

In this lab you'll be working with the *The Lending Club* data. 

- Import the data available in the file `'loan_final.csv'` 
- Drop rows with missing values in the `'total_pymnt'` column (this is your target column) 
- Print the first five rows of the data 
- Print the dimensions of the data 

In [5]:
# Import the data
data = pd.read_csv('loan_final.csv')
data.head(3).append(data.tail(3))

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
42535,2525.0,225.0,36 months,9.33%,80.69,B,< 1 year,RENT,110000.0,Not Verified,Does not meet the credit policy. Status:Fully ...,other,NY,NaN,2904.498829,Individual
42536,6500.0,0.0,36 months,8.38%,204.84,A,< 1 year,NONE,NaN,Not Verified,Does not meet the credit policy. Status:Fully ...,other,NY,NaN,7373.904962,Individual
42537,5000.0,0.0,36 months,7.75%,156.11,A,10+ years,MORTGAGE,70000.0,Not Verified,Does not meet the credit policy. Status:Fully ...,other,CT,NaN,5619.762090,Individual


In [8]:
# Drop rows with no target value
data.dropna(subset=['total_pymnt'], inplace=True)

# Print the first five rows
data.head()

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual


In [10]:
# Print the dimensions of data 
data.shape

(42535, 16)

## Generating a Hold Out Test Set

While we will be using K-fold cross validation to select an optimal model, we still want a final hold out test set that is completely independent of any modeling decisions. As such, pull out a sample of 30% of the total available data. For consistency of results, use random seed 42. 

In [16]:
# Features to build the model
features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc', 
            'home_ownership', 'verification_status', 'emp_length']

X = data[features]
y = data[['total_pymnt']] # Target that we are working towards

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Preprocessing (Numerical features) 

- Fill all missing values in numeric features with their respective means 
- Standardize all the numeric features  
- Convert the final results into DataFrames 

In [17]:
# Select continuous features
cont_features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc']
X_train_cont = X_train.loc[:, cont_features]
X_test_cont = X_test.loc[:, cont_features]

# Instantiate SimpleImputer - fill the missing values with the mean
si = SimpleImputer(strategy='mean')

# Fit and transform the training data
X_train_imputed = si.fit_transform(X_train_cont)

# Transform test data
X_test_imputed = si.transform(X_test_cont)

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform the training data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_imputed), columns=cont_features)

# Transform test data
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_imputed), columns=cont_features)

## Preprocessing (Categorical features) 

- Fill all missing values in categorical features with the string `'missing'` 
- One-hot encode all categorical features 
- Convert the final results into DataFrames 


In [19]:
# Select only the categorical features
cat_features = ['home_ownership', 'verification_status', 'emp_length']
X_train_cat = X_train.loc[:, cat_features]
X_test_cat = X_test.loc[:, cat_features]

# Fill missing values with the string 'missing'

X_train_cat.fillna(value='missing', inplace=True)
X_test_cat.fillna(value='missing', inplace=True)

# OneHotEncode categorical variables
ohe = OneHotEncoder(handle_unknown='ignore')

# Transform training and test sets
X_train_ohe = ohe.fit_transform(X_train_cat)
X_test_ohe = ohe.transform(X_test_cat)

# Get all categorical feature names
cat_columns = ohe.get_feature_names(input_features=X_train_cat.columns)

# Fit and transform the training data
X_train_categorical = pd.DataFrame(X_train_ohe.todense(), columns=cat_columns)

# Transform test data
X_test_categorical = pd.DataFrame(X_test_ohe.todense(), columns=cat_columns)

Run the below cell to combine the numeric and categorical features. 

In [20]:
# Combine continuous and categorical feature DataFrames
X_train_all = pd.concat([X_train_scaled, X_train_categorical], axis=1)
X_test_all = pd.concat([X_test_scaled, X_test_categorical], axis=1)

# Number of input features
n_features = X_train_all.shape[1]

- Standardize the target DataFrames (`y_train` and `y_test`) 

In [21]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform Y (train)
y_train_scaled = ss_y.fit_transform(y_train)

# Transform test Y (test)
y_test_scaled = ss_y.transform(y_test)

In [27]:
print(type(X_train_scaled))
print(type(X_test_scaled))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [29]:
display('X_train_scaled Dataset:', X_train_scaled.head(3))
print('=======================================')
display('X_test_scaled Dataset:', X_test_scaled.head(3))

'X_train_scaled Dataset:'

,loan_amnt,funded_amnt_inv,installment,annual_inc
0,0.961300,0.487971,-0.013952,-0.326925
1,1.741417,-0.336399,0.812258,0.582909
2,-0.487489,-0.374986,-0.316000,-0.143161


'X_test_scaled Dataset:'

,loan_amnt,funded_amnt_inv,installment,annual_inc
0,-0.082234,0.045969,-0.344609,-0.409545
1,1.065991,1.088323,0.430134,-0.380439
2,0.390564,0.530067,-0.123644,-0.301346


In [30]:
print('X_train_scaled shape:', X_train_scaled.shape)
print('X_test_scaled shape:', X_test_scaled.shape)

X_train_scaled shape: (29774, 4)
X_test_scaled shape: (12761, 4)


## Define a K-fold Cross Validation Methodology

Now that your have a complete holdout test set, you will perform k-fold cross-validation using the following steps: 

- Create a function that returns a compiled deep learning model 
- Use the wrapper function `KerasRegressor()` that defines how these folds are trained 
- Call the `cross_val_predict()` function to perform k-fold cross-validation 

In the cell below, we've defined a baseline model that returns a compiled Keras models. 

In [31]:
# Define a function that returns a compiled Keras model 
def create_baseline_model():
    
    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(10, activation='relu', input_shape=(n_features,)))

    # Second hidden layer
    model.add(layers.Dense(5, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model

Wrap `create_baseline_model` inside a call to `KerasRegressor()`, and: 

- Train for 150 epochs 
- Set the batch size to 256 

> NOTE: Refer to the [documentation](https://keras.io/scikit-learn-api/) to learn about `KerasRegressor()`.  

In [33]:
# Wrap the above function for use in cross-validation
keras_wrapper_1 = KerasRegressor(create_baseline_model,
                                epochs=150,
                                batch_size=256,
                                verbose=0)

Use `cross_val_predict()` to generate cross-validated predictions with: 
- 5-fold cv 
- scaled input (`X_train_all`) and output (`y_train_scaled`) 

In [34]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_baseline_preds = cross_val_predict(keras_wrapper_1, X_train_all, y_train_scaled, cv=5)

- Find the RMSE on train data 

In [35]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(y_train_scaled, cv_baseline_preds))

0.4442127693659362

- Convert the scaled predictions back to original scale 
- Calculate RMSE in the original units with `y_train` and `baseline_preds` 

In [36]:
# Convert the predictions back to original scale
baseline_preds = ss_y.inverse_transform(cv_baseline_preds)

# RMSE on train data (original scale)
np.sqrt(mean_squared_error(y_train, baseline_preds))

4046.1295634002836

## Intentionally Overfitting a Model

Now that you've developed a baseline model, its time to intentionally overfit a model. To overfit a model, you can:
* Add layers
* Make the layers bigger
* Increase the number of training epochs

Again, be careful here. Think about the limitations of your resources, both in terms of your computers specs and how much time and patience you have to let the process run. Also keep in mind that you will then be regularizing these overfit models, meaning another round of experiments and more time and resources.

In [63]:
# Define a function that returns a compiled Keras model 
def create_bigger_model():
    
    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(10, activation='relu', input_shape=(n_features,)))

    # Second hidden layer
    model.add(layers.Dense(10, activation='relu'))
    
    # Third hidden layer
    model.add(layers.Dense(10, activation='relu'))
    
    # Fourth hidden layer
    model.add(layers.Dense(10, activation='relu'))
    
    # Fifth hidden layer
    model.add(layers.Dense(10, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model

In [64]:
# Wrap the above function for use in cross-validation
keras_wrapper_2 = KerasRegressor(create_bigger_model,  
                                 epochs=150, 
                                 batch_size=256, 
                                 verbose=0)

In [65]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_bigger_model_preds = cross_val_predict(keras_wrapper_2, X_train_all, y_train_scaled, cv=5)


In [66]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(y_train_scaled, cv_bigger_model_preds))

0.44725946672619843

## Regularizing the Model to Achieve Balance  

Now that you have a powerful model (albeit an overfit one), we can now increase the generalization of the model by using some of the regularization techniques we discussed. Some options you have to try include:  
* Adding dropout
* Adding L1/L2 regularization
* Altering the layer architecture (add or remove layers similar to above)  

This process will be constrained by time and resources. Be sure to test at least two different methodologies, such as dropout and L2 regularization. If you have the time, feel free to continue experimenting. 

In [67]:
# Define a function that returns a compiled Keras model 
def create_regularized_model():
    
    # I will start with the Dropout
    
    # Initialize model
    model = models.Sequential()

    # Input layer with dropout
    model.add(layers.Dropout(0.3, input_shape=(n_features,)))
    
    # First hidden layer
    model.add(layers.Dense(10, kernel_regularizer=regularizers.l2(0.005), activation='relu'))
    model.add(layers.Dropout(0.3))

    # Second hidden layer
    model.add(layers.Dense(10, kernel_regularizer=regularizers.l2(0.005), activation='relu'))
    model.add(layers.Dropout(0.3))
    
    # Third hidden layer
    model.add(layers.Dense(10, kernel_regularizer=regularizers.l2(0.005), activation='relu'))
    model.add(layers.Dropout(0.3))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model
    
    

In [72]:
# Wrap the above function for use in cross-validation
keras_wrapper_3 = KerasRegressor(create_regularized_model,  
                                 epochs=150, 
                                 batch_size=256, 
                                 verbose=0)

In [73]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_dropout_preds = cross_val_predict(keras_wrapper_3, X_train_all, y_train_scaled, cv=5)

In [75]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(y_train_scaled, cv_dropout_preds))

0.5900710911438259

## Final Evaluation

Now that you have selected a network architecture, tested various regularization procedures and tuned hyperparameters via a validation methodology, it is time to evaluate your final model on the test set. Fit the model using all of the training data using the architecture and hyperparameters that were most effective in your experiments above. Afterwards, measure the overall performance on the hold-out test data which has been left untouched (and hasn't leaked any data into the modeling process)! 

In [77]:
# ⏰ This cell may take several mintes to run

# Initialize model
model = models.Sequential()

# First hidden layer
model.add(layers.Dense(10, activation='relu', input_shape=(n_features,)))

# Second hidden layer
model.add(layers.Dense(5, activation='relu'))

# Output layer
model.add(layers.Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='SGD', loss='mse', metrics=['mse'])

# Train the model
model.fit(X_train_all, y_train_scaled, epochs=150, batch_size=256, verbose=0)

In [79]:
# Generate predictions on test data
final_preds_scaled = model.predict(X_test_all)

# Convert the predictions back to the original scale
final_preds = ss_y.inverse_transform(final_preds_scaled)

# RMSE on test data with the original scale
np.sqrt(mean_squared_error(y_test, final_preds))

3852.3752838266923

## Summary

In this lab, you investigated some data from *The Lending Club* in a complete data science pipeline to build neural networks with good performance. You began with reserving a hold-out set for testing which never was touched during the modeling phase. From there, you implemented a k-fold cross validation methodology in order to assess an initial baseline model and various regularization methods. 